In [1]:
from transformers import AutoTokenizer, LxmertForQuestionAnswering
import torch
import json

c:\Users\opoob\miniconda3\envs\cse493\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained('unc-nlp/lxmert-vqa-uncased')
model = LxmertForQuestionAnswering.from_pretrained('unc-nlp/lxmert-vqa-uncased')

In [6]:
# Language features
# https://huggingface.co/docs/transformers/main_classes/tokenizer
input_questions = json.load(open('../Questions/v3_OpenEnded_mscoco_val2014_questions.json'))
input_questions = [q['question'] for q in input_questions['questions']]
tokens = tokenizer(
    input_questions, 
    padding="max_length", 
    max_length=20, 
    truncation=True, 
    return_token_type_ids=True, # ?
    return_attention_mask=True, # ?
    add_special_tokens=True, # ?
    return_tensors='pt')
inputs = tokens['input_ids']

{'input_ids': tensor([[ 101, 2003, 2045,  ...,    0,    0,    0],
        [ 101, 2024, 2122,  ...,    0,    0,    0],
        [ 101, 2024, 2122,  ...,    0,    0,    0],
        ...,
        [ 101, 2054, 2193,  ...,    0,    0,    0],
        [ 101, 2054, 2003,  ...,    0,    0,    0],
        [ 101, 2003, 1996,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [ ]:
# Visual features